<a href="https://colab.research.google.com/github/odus05/Image_Retrieval/blob/master/cats_and_dogs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import matplotlib.pyplot as plt
import os, shutil
import cv2
from keras.models import Model, Sequential, load_model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import adam
from keras import backend as K

plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12
plt.rcParams['axes.unicode_minus'] = False
%matplotlib inline

Using TensorFlow backend.


In [0]:
from google.colab import drive

drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
### 1. Data Load

In [0]:
# 원본 데이터셋을 압축 해제한 디렉터리 경로
original_dataset_cat_dir = '/content/gdrive/My Drive/Colab Notebooks/datasets/cats_and_dogs/Cat'
original_dataset_dog_dir = '/content/gdrive/My Drive/Colab Notebooks/datasets/cats_and_dogs/Dog'

# 소규모 데이터셋을 저장할 디렉터리
base_dir = '/content/gdrive/My Drive/Colab Notebooks/datasets/cats_and_dogs_small'
if os.path.exists(base_dir):  # 반복적인 실행을 위해 디렉토리를 삭제합니다.
    shutil.rmtree(base_dir)   # 이 코드는 책에 포함되어 있지 않습니다.
os.mkdir(base_dir)

# 훈련, 검증, 테스트 분할을 위한 디렉터리
train_dir = os.path.join(base_dir, 'train')
os.mkdir(train_dir)
validation_dir = os.path.join(base_dir, 'validation')
os.mkdir(validation_dir)
test_dir = os.path.join(base_dir, 'test')
os.mkdir(test_dir)

# 훈련용 고양이 사진 디렉터리
train_cats_dir = os.path.join(train_dir, 'cats')
os.mkdir(train_cats_dir)

# 훈련용 강아지 사진 디렉터리
train_dogs_dir = os.path.join(train_dir, 'dogs')
os.mkdir(train_dogs_dir)

# 검증용 고양이 사진 디렉터리
validation_cats_dir = os.path.join(validation_dir, 'cats')
os.mkdir(validation_cats_dir)

# 검증용 강아지 사진 디렉터리
validation_dogs_dir = os.path.join(validation_dir, 'dogs')
os.mkdir(validation_dogs_dir)

# 테스트용 고양이 사진 디렉터리
test_cats_dir = os.path.join(test_dir, 'cats')
os.mkdir(test_cats_dir)

# 테스트용 강아지 사진 디렉터리
test_dogs_dir = os.path.join(test_dir, 'dogs')
os.mkdir(test_dogs_dir)

# 처음 1,000개의 고양이 이미지를 train_cats_dir에 복사합니다
try:
  fnames = ['cat.{}.jpg'.format(i+1) for i in range(1000)]
  for fname in fnames:
      src = os.path.join(original_dataset_cat_dir, fname)
      dst = os.path.join(train_cats_dir, fname)
      shutil.copyfile(src, dst)

  # 다음 500개 고양이 이미지를 validation_cats_dir에 복사합니다
  fnames = ['cat.{}.jpg'.format(i+1) for i in range(1000, 1500)]
  for fname in fnames:
      src = os.path.join(original_dataset_cat_dir, fname)
      dst = os.path.join(validation_cats_dir, fname)
      shutil.copyfile(src, dst)
      
  # 다음 500개 고양이 이미지를 test_cats_dir에 복사합니다
  fnames = ['cat.{}.jpg'.format(i+1) for i in range(1500, 2000)]
  for fname in fnames:
      src = os.path.join(original_dataset_cat_dir, fname)
      dst = os.path.join(test_cats_dir, fname)
      shutil.copyfile(src, dst)
      
  # 처음 1,000개의 강아지 이미지를 train_dogs_dir에 복사합니다
  fnames = ['dog.{}.jpg'.format(i+1) for i in range(1000)]
  for fname in fnames:
      src = os.path.join(original_dataset_dog_dir, fname)
      dst = os.path.join(train_dogs_dir, fname)
      shutil.copyfile(src, dst)
      
  # 다음 500개 강아지 이미지를 validation_dogs_dir에 복사합니다
  fnames = ['dog.{}.jpg'.format(i+1) for i in range(1000, 1500)]
  for fname in fnames:
      src = os.path.join(original_dataset_dog_dir, fname)
      dst = os.path.join(validation_dogs_dir, fname)
      shutil.copyfile(src, dst)
      
  # 다음 500개 강아지 이미지를 test_dogs_dir에 복사합니다
  fnames = ['dog.{}.jpg'.format(i+1) for i in range(1500, 2000)]
  for fname in fnames:
      src = os.path.join(original_dataset_dog_dir, fname)
      dst = os.path.join(test_dogs_dir, fname)
      shutil.copyfile(src, dst)
except Exception as e:
  print(e)

In [0]:
print('훈련용 고양이 이미지 전체 개수:', len(os.listdir(train_cats_dir)))
print('훈련용 강아지 이미지 전체 개수:', len(os.listdir(train_dogs_dir)))
print('검증용 고양이 이미지 전체 개수:', len(os.listdir(validation_cats_dir)))
print('검증용 강아지 이미지 전체 개수:', len(os.listdir(validation_dogs_dir)))
print('테스트용 고양이 이미지 전체 개수:', len(os.listdir(test_cats_dir)))
print('테스트용 강아지 이미지 전체 개수:', len(os.listdir(test_dogs_dir)))

훈련용 고양이 이미지 전체 개수: 1000
훈련용 강아지 이미지 전체 개수: 1000
검증용 고양이 이미지 전체 개수: 500
검증용 강아지 이미지 전체 개수: 500
테스트용 고양이 이미지 전체 개수: 500
테스트용 강아지 이미지 전체 개수: 500


In [0]:
images = []
for root, _, files in os.walk(train_dir):
    for fname in files[:3]:
        fname = os.path.join(root, fname)
        img = cv2.imread(fname, cv2.IMREAD_COLOR)
        img = np.asarray(img, dtype='float32') / 255
        print('fname: {} | img.shape: {}'.format(fname, img.shape))
        img = cv2.resize(img, (150, 150))
        images.append(img)

fname: /content/gdrive/My Drive/Colab Notebooks/datasets/cats_and_dogs_small/train/cats/cat.1.jpg | img.shape: (281, 300, 3)
fname: /content/gdrive/My Drive/Colab Notebooks/datasets/cats_and_dogs_small/train/cats/cat.2.jpg | img.shape: (397, 312, 3)
fname: /content/gdrive/My Drive/Colab Notebooks/datasets/cats_and_dogs_small/train/cats/cat.3.jpg | img.shape: (415, 500, 3)
fname: /content/gdrive/My Drive/Colab Notebooks/datasets/cats_and_dogs_small/train/dogs/dog.1.jpg | img.shape: (500, 327, 3)
fname: /content/gdrive/My Drive/Colab Notebooks/datasets/cats_and_dogs_small/train/dogs/dog.2.jpg | img.shape: (199, 188, 3)
fname: /content/gdrive/My Drive/Colab Notebooks/datasets/cats_and_dogs_small/train/dogs/dog.3.jpg | img.shape: (375, 500, 3)


In [0]:
K.clear_session()

model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer=adam(lr=1e-4),
              metrics=['acc'])

model.summary()







Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 34, 34, 128)       73856     
___________________________________

In [0]:
%%time
# 모든 이미지를 1/255로 스케일을 조정합니다
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,)

# 검증 데이터는 증식되어서는 안 됩니다!
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        # 타깃 디렉터리
        train_dir,
        # 모든 이미지를 150 × 150 크기로 바꿉니다
        target_size=(150, 150),
        batch_size=32,
        # binary_crossentropy 손실을 사용하기 때문에 이진 레이블을 만들어야 합니다
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=32,
        class_mode='binary')

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
CPU times: user 73.1 ms, sys: 2 ms, total: 75.1 ms
Wall time: 219 ms


In [0]:
data_batch, labels_batch = next(train_generator)
print('배치 데이터 크기:', data_batch.shape)
print('배치 레이블 크기:', labels_batch.shape)

배치 데이터 크기: (32, 150, 150, 3)
배치 레이블 크기: (32,)


In [0]:
history = model.fit_generator(
      train_generator,
      steps_per_epoch=100,  # 3,200개의 샘플을 처리할 때까지, 100개의 배치를 뽑는다.
      epochs=100,
      validation_data=validation_generator,
      validation_steps=50)
model.save('/content/gdrive/My Drive/Colab Notebooks/datasets/cats_and_dogs_small/cats_and_dogs_small_1.h5')




Epoch 1/100





100/100 [==============================] - 44s 436ms/step - loss: 0.6929 - acc: 0.5181 - val_loss: 0.6892 - val_acc: 0.5000
Epoch 2/100
100/100 [==============================] - 36s 355ms/step - loss: 0.6806 - acc: 0.5594 - val_loss: 0.6711 - val_acc: 0.5625
Epoch 3/100
100/100 [==============================] - 35s 352ms/step - loss: 0.6640 - acc: 0.5841 - val_loss: 0.6436 - val_acc: 0.6009
Epoch 4/100
100/100 [==============================] - 35s 353ms/step - loss: 0.6497 - acc: 0.6178 - val_loss: 0.6256 - val_acc: 0.6211
Epoch 5/100
100/100 [==============================] - 34s 342ms/step - loss: 0.6293 - acc: 0.6409 - val_loss: 0.6622 - val_acc: 0.6199
Epoch 6/100
100/100 [==============================] - 38s 379ms/step - loss: 0.6031 - acc: 0.6722 - val_loss: 0.5809 - val_acc: 0.6778
Epoch 7/100
100/100 [==============================] - 35s 353ms/step - loss: 0.5942 - acc: 0.6835 - val_loss: 0.5843 - val_acc: 0.6827
Epoch 8/100
100/100 [===================

In [0]:
def show_history(history):
  acc = history.history['acc']
  val_acc = history.history['val_acc']
  loss = history.history['loss']
  val_loss = history.history['val_loss']

  epochs = range(len(acc))

  plt.plot(epochs, acc, 'bo', label='Training acc')
  plt.plot(epochs, val_acc, 'b', label='Validation acc')
  plt.title('Training and validation accuracy')
  plt.legend()

  plt.figure()

  plt.plot(epochs, loss, 'bo', label='Training loss')
  plt.plot(epochs, val_loss, 'b', label='Validation loss')
  plt.title('Training and validation loss')
  plt.legend()

  plt.show()

In [0]:
show_history(history)

데이터가 적기 때문에 컨브넷을 처음부터 훈련해서 더 높은 정확도를 달성하기는 어렵습니다. 이런 상황에서 정확도를 높이기 위한 다음 단계는 사전 훈련된 모델을 사용하는 것입니다.

In [0]:
### 특성 추출(Feature Extraction) -> Transfer Learning

In [0]:
from keras.applications import VGG16

conv_base = VGG16(weights='imagenet', 
                  include_top=False,
                  input_shape=(150, 150, 3))
conv_base.summary()

In [0]:
### 1) 데이터 증식을 사용하지 않는 빠른 특성추출
### - conv_base에 데이터를 주입하고 출력을 저장하고 이를 새로운 모델의 입력에 사용합니다.

In [0]:
datagen = ImageDataGenerator(rescale=1./255)
batch_size = 20

def extract_features(directory, sample_count):
    features = np.zeros(shape=(sample_count, 4, 4, 512))
    labels = np.zeros(shape=(sample_count))
    generator = datagen.flow_from_directory(
        directory,
        target_size=(150, 150), 
        batch_size=batch_size, 
        class_mode='binary')
    
    for i, (inputs_batch, labels_batch) in enumerate(generator):
        features_batch = conv_base.predict(inputs_batch)
        features[i * batch_size : (i+1) * batch_size] = features_batch
        labels[i * batch_size : (i+1) * batch_size] = labels_batch
        
        if (i+1) * batch_size >= sample_count:
            # 제너레이터는 루프 안에서 무한하게 데이터를 만들어내므로 
            # 모든 이미지를 한 번씩 처리하고 나면 중지합니다
            break
    
    return features, labels

In [0]:
train_features, train_labels = extract_features(train_dir, 2000)
validation_features, validation_labels = extract_features(validation_dir, 1000)
test_features, test_labels = extract_features(test_dir, 1000)

In [0]:
print('train_features      : {}, train_labels      : {}'.format(train_features.shape, train_labels.shape))
print('validation_features : {}, validation_labels : {}'.format(validation_features.shape, validation_labels.shape))
print('test_features       : {}, test_labels : {}'.format(test_features.shape, test_labels.shape))

In [0]:
train_features = np.reshape(train_features, (-1, 4*4*512))
validation_features = validation_features.reshape([-1, 4*4*512])
test_features = np.reshape(test_features, (-1, 4*4*512))

In [0]:
model = Sequential()
model.add(Dense(256, activation='relu', input_dim=4*4*512))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.summary()

In [0]:
model.compile(optimizer=adam(lr=2e-5),
              loss='binary_crossentropy',
              metrics=['acc'])
history = model.fit(train_features, train_labels,
                    epochs=30,
                    batch_size=20,
                    validation_data = (validation_features, validation_labels))
model.save('/content/gdrive/My Drive/Colab Notebooks/datasets/cats_and_dogs_small/cats_and_dogs_small_2.h5')
show_history(history)

약 90%의 검증 정확도에 도달했습니다. 이전 절에서 처음부터 훈련시킨 작은 모델에서 얻은 것보다 훨씬 좋습니다. 하지만 이 그래프는 많은 비율로 드롭아웃을 사용했음에도 불구하고 훈련이 시작하면서 거의 바로 과대적합되고 있다는 것을 보여줍니다. 작은 이미지 데이터셋에서는 과대적합을 막기 위해 필수적인 데이터 증식을 사용하지 않았기 때문입니다.

In [0]:
### 2) 데이터 증식을 사용한 특성 추출
### - conv_base 모델을 확장하고 입력데이터를 사용해 End-to-End로 실행!!

In [0]:
model = Sequential()
model.add(conv_base)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

In [0]:
print('conv_base를 동결하기 전 훈련되는 가중치의 수:', 
      len(model.trainable_weights))

# VGG16 Weight Freezing
conv_base.trainable = False

print('conv_base를 동결한 후 훈련되는 가중치의 수:', 
      len(model.trainable_weights))

In [0]:
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=20,
      width_shift_range=0.1,
      height_shift_range=0.1,
      shear_range=0.1,
      zoom_range=0.1,
      horizontal_flip=True,
      fill_mode='nearest')

# 검증 데이터는 증식되어서는 안 됩니다!
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        # 타깃 디렉터리
        train_dir,
        # 모든 이미지의 크기를 150 × 150로 변경합니다
        target_size=(150, 150),
        batch_size=20,
        # binary_crossentropy 손실을 사용하므로 이진 레이블이 필요합니다
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')

model.compile(loss='binary_crossentropy',
              optimizer=adam(lr=2e-5),
              metrics=['acc'])

history = model.fit_generator(
      train_generator,
      steps_per_epoch=100,
      epochs=30,
      validation_data=validation_generator,
      validation_steps=50,
      verbose=1)
model.save('/content/gdrive/My Drive/Colab Notebooks/datasets/cats_and_dogs_small/cats_and_dogs_small_3.h5')
show_history(history)

검증 정확도가 이전과 비슷하지만 처음부터 훈련시킨 소규모 컨브넷보다 과대적합이 줄었습니다.

In [0]:
### 미세 조정(Fine-tuning)
### - 미세 조정은 특성 추출에 사용했던 동결 모델의 상위 층 몇 개를 동결에서 해제하고 모델에 새로 추가한 층(여기서는 완전 연결 분류기)과 함께 훈련 하는 것입니다.

### 미세 조정 방법 ###
###1. 사전에 훈련된 기반 네트워크 위에 새로운 네트워크를 추가합니다.
###2. 기반 네트워크를 동결합니다.
###3. 새로 추가한 네트워크를 훈련합니다.
###4. 기반 네트워크에서 일부 층의 동결을 해제합니다.
###5. 동결을 해제한 층과 새로 추가한 층을 함께 훈련합니다.

In [0]:
### Unfreezing
conv_base.trainable = True
set_trainable = False

for layer in conv_base.layers:
  if layer.name == 'block5_conv1':
    set_trainable = True
  if set_trainable:
    layer.trainable = True
  else:
    layer.trainable = False

In [0]:
### 학습률을 낮추는 이유는 미세 조정하는 세 개의 층에서 학습된 표현을 조금씩 수정하기 위해서입니다. 변경량이 너무 크면 학습된 표현에 나쁜 영향을 끼칠 수 있습니다.
model.compile(loss='binary_crossentropy',
              optimizer=adam(lr=1e-5),
              metrics=['acc'])
model.summary()
history = model.fit_generator(
    train_generator,
    steps_per_epoch=100,
    epochs=100,
    validation_data = validation_generator,
    validation_steps=50)
model.save('/content/gdrive/My Drive/Colab Notebooks/datasets/cats_and_dogs_small/cats_and_dogs_small_4.h5')
show_history(history)

In [0]:
### 그래프가 불규칙하게 보이기 때문에 지수 이동 평균으로 정확도와 손실값을 부드럽게 표현합니다.

In [0]:
def smooth_curve(points, factor=0.8):
    smoothed_points = []
    for point in points:
        if smoothed_points:
            previous = smoothed_points[-1]
            smoothed_points.append(previous * factor + point * (1 - factor))
        else:
            smoothed_points.append(point)
    return smoothed_points


acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs,
         smooth_curve(acc), 'bo', label='Smoothed training acc')
plt.plot(epochs,
         smooth_curve(val_acc), 'b', label='Smoothed validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs,
         smooth_curve(loss), 'bo', label='Smoothed training loss')
plt.plot(epochs,
         smooth_curve(val_loss), 'b', label='Smoothed validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [0]:
test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')

test_loss, test_acc = model.evaluate_generator(test_generator, steps=50)
print('test acc:', test_acc)


<정리>
컨브넷은 컴퓨터 비전 작업에 가장 뛰어난 머신 러닝 모델입니다. 아주 작은 데이터셋에서도 처음부터 훈련해서 괜찮은 성능을 낼 수 있습니다.
작은 데이터셋에서는 과대적합이 큰 문제입니다. 데이터 증식은 이미지 데이터를 다룰 때 과대적합을 막을 수 있는 강력한 방법입니다.
특성 추출 방식으로 새로운 데이터셋에 기존의 컨브넷을 쉽게 재사용할 수 있습니다. 작은 이미지 데이터셋으로 작업할 때 효과적인 기법입니다.
특성 추출을 보완하기 위해 미세 조정을 사용할 수 있습니다. 미세 조정은 기존 모델에서 사전에 학습한 표현의 일부를 새로운 문제에 적응시킵니다. 이 기법은 조금 더 성능을 끌어올립니다.

In [0]:
### 컨브넷 학습 시각화 ###
###  컨브넷의 표현은 시각적인 개념을 학습한 것이기 때문에 시각화하기 아주 좋습니다.
### 1) 컨브넷 중간 층의 출력(중간 층에 있는 활성화)을 시각화하기 : 연속된 컨브넷층이 입력을 어떻게 변형시키는지 이해하고 개별적인 컨브넷 필터의 의미를 파악하는 데 도움이 됩니다.
### 2) 컨브넷 필터를 시각화하기 : 컨브넷의 필터가 찾으려는 시각적인 패턴과 개념이 무엇인지 상세하게 이해하는 데 도움이 됩니다.
### 3) 클래스 활성화에 대한 히트맵을 이미지에 시각화하기 : 이미지의 어느 부분이 주어진 클래스에 속하는 데 기여했는지 이해하고 이미지에서 객체의 위치를 추정하는 데 도움이 됩니다.

In [0]:
### 1) 컨브넷 중간 층의 활성화 시각화하기

In [0]:
model = load_model('/content/gdrive/My Drive/Colab Notebooks/datasets/cats_and_dogs_small/cats_and_dogs_small_1.h5')
model.summary()

In [0]:
from keras.preprocessing import image

img_path = '/content/gdrive/My Drive/Colab Notebooks/datasets/cats_and_dogs_small/test/cats/cat.1500.jpg'
img = image.load_img(img_path, target_size=(150, 150))
img_tensor = image.img_to_array(img)
print("img_tensor's shape (before expand_dims) :", img_tensor.shape)

img_tensor = np.expand_dims(img_tensor, axis=0)
print("img_tensor's shape (after expand_dims) :", img_tensor.shape)

# 모델이 훈련될 때 입력에 적용한 전처리 방식을 동일하게 사용합니다
img_tensor /= 255.
# 이미지 텐서의 크기는 (1, 150, 150, 3)입니다
print(img_tensor.shape)

plt.imshow(img_tensor[0])
plt.show()

In [0]:
# 상위 8개의 층 출력을 추출  (Flatten 전까지)
layer_outputs = [layer.output for layer in model.layers[:8]]
# 입력에 대해 8개 층의 출력을 반환하는 모델을 만듦
activation_model = Model(inputs=model.input, outputs=layer_outputs)
activation_model.summary()
# 이 모델은 하나의 입력(이미지)과 층의 활성화마다 총 8개의 출력을 가집니다.
# 층의 활성화마다 하나씩 8개의 넘파이 배열로 이루어진 리스트를 반환한다.
activations = activation_model.predict(img_tensor)
print(len(activations))

In [0]:
first_layer_activation = activations[0]
print(first_layer_activation.shape)

In [0]:
### 32개의 채널을 가진 148 × 148 크기의 특성 맵입니다. 원본 모델의 첫 번째 층의 활성화 중에서 1 ~ 5 번째 채널을 그려 보겠습니다:
fig, axes = plt.subplots(1, 5, figsize=(14,6))
pl_images = [
             first_layer_activation[0,:,:,0],
             first_layer_activation[0,:,:,1],
             first_layer_activation[0,:,:,2],
             first_layer_activation[0,:,:,3],
             first_layer_activation[0,:,:,4]
]
for i, image in enumerate(pl_images):
  axes[i].imshow(image, cmap='viridis')

In [0]:
# 층의 이름을 그래프 제목으로 사용합니다
layer_names = []
for layer in model.layers[:8]:
    layer_names.append(layer.name)

images_per_row = 16

# 특성 맵을 그립니다
for layer_name, layer_activation in zip(layer_names, activations):
    # 특성 맵에 있는 특성의 수
    n_features = layer_activation.shape[-1]

    # 특성 맵의 크기는 (1, size, size, n_features)입니다
    size = layer_activation.shape[1]

    # 활성화 채널을 위한 그리드 크기를 구합니다
    n_cols = n_features // images_per_row
    display_grid = np.zeros((size * n_cols, images_per_row * size))

    # 각 활성화를 하나의 큰 그리드에 채웁니다
    for col in range(n_cols):
        for row in range(images_per_row):
            channel_image = layer_activation[0,
                                             :, :,
                                             col * images_per_row + row]
            # 그래프로 나타내기 좋게 특성을 처리합니다
            channel_image -= channel_image.mean()
            channel_image /= channel_image.std()
            channel_image *= 64
            channel_image += 128
            channel_image = np.clip(channel_image, 0, 255).astype('uint8')
            display_grid[col * size : (col + 1) * size,
                         row * size : (row + 1) * size] = channel_image

    # 그리드를 출력합니다
    scale = 1. / size
    plt.figure(figsize=(scale * display_grid.shape[1],
                        scale * display_grid.shape[0]))
    plt.title(layer_name)
    plt.grid(False)
    plt.imshow(display_grid, aspect='auto', cmap='viridis')

plt.show()

In [0]:
### 컨브넷 필터 시각화 하기!!
### 특정 합성곱 층의 한 필터의 값을 최대화하는 손실 함수를 정의합니다. 이 활성화 값을 최대화하기 위해 입력 이미지를 변경하도록 확률적 경사 상승법을 사용합니다.
### ImageNet에 사전 훈련된 VGG16 네트워크에서 block3_conv1 층의 필터 0번의 활성화를 손실로 정의합니다

In [0]:
from keras import backend as K

model = VGG16(weights='imagenet',
              include_top=False)

layer_name = 'block3_conv1'
filter_index = 0

layer_output = model.get_layer(layer_name).output
loss = K.mean(layer_output[:, :, :, filter_index])

In [0]:
### 경사 상승법을 구현하기 위해 모델의 입력에 대한 손실의 그래디언트가 필요합니다. 이를 위해 케라스의 backend 모듈에 있는 gradients 함수를 사용하겠습니다:
# gradients 함수가 반환하는 텐서 리스트(여기에서는 크기가 1인 리스트)에서 첫 번째 텐서를 추출합니다
grads = K.gradients(loss, model.input)[0]

In [0]:
### 경사 상승법 과정을 부드럽게 하기 위해 사용하는 한 가지 기법은 그래디언트 텐서를 L2 노름(텐서에 있는 값을 제곱합의 제곱근)으로 나누어 정규화하는 것입니다. 이렇게 하면 입력 이미지에 적용할 수정량의 크기를 항상 일정 범위 안에 놓을 수 있습니다.
# 0 나눗셈을 방지하기 위해 1e–5을 더합니다
grads /= (K.sqrt(K.mean(K.square(grads))) + 1e-5)

In [0]:
### 이제 주어진 입력 이미지에 대해 손실 텐서와 그래디언트 텐서를 계산해야 합니다. 케라스 백엔드 함수를 사용하여 처리하겠습니다. iterate는 넘파이 텐서(크기가 1인 텐서의 리스트)를 입력으로 받아 손실과 그래디언트 두 개의 넘파이 텐서를 반환합니다.
iterate = K.function([model.input], [loss, grads])

# 테스트:
import numpy as np
loss_value, grads_value = iterate([np.zeros((1, 150, 150, 3))])

In [0]:
### 여기에서 파이썬 루프를 만들어 확률적 경사 상승법을 구성합니다:
# 잡음이 섞인 회색 이미지로 시작합니다
input_img_data = np.random.random((1, 150, 150, 3)) * 20 + 128.

# 업데이트할 그래디언트의 크기
step = 1.
for i in range(40):   # 경사 상승법을 40회 실행합니다
    # 손실과 그래디언트를 계산합니다
    loss_value, grads_value = iterate([input_img_data])
    # 손실을 최대화하는 방향으로 입력 이미지를 수정합니다
    input_img_data += grads_value * step

In [0]:
### 결과 이미지 텐서는 (1, 150, 150, 3) 크기의 부동 소수 텐서입니다. 이 텐서의 값은 [0, 255] 사이의 정수가 아닙니다. 따라서 출력 가능한 이미지로 변경하기 위해 후처리할 필요가 있습니다. 이를 위해 간단한 함수를 정의해 사용하겠습니다:
def deprocess_image(x):
    # 텐서의 평균이 0, 표준 편차가 0.1이 되도록 정규화합니다
    x -= x.mean()
    x /= (x.std() + 1e-5)
    x *= 0.1

    # [0, 1]로 클리핑합니다
    x += 0.5
    x = np.clip(x, 0, 1)

    # RGB 배열로 변환합니다
    x *= 255
    x = np.clip(x, 0, 255).astype('uint8')
    return x

def generate_pattern(layer_name, filter_index, size=150):
  # 주어진 층과 필터의 활성화를 최대화하기 위한 손실 함수를 정의합니다
  layer_output = model.get_layer(layer_name).output
  loss = K.mean(layer_output[:, :, :, filter_index])

  # 손실에 대한 입력 이미지의 그래디언트를 계산합니다
  grads = K.gradients(loss, model.input)[0]

  # 그래디언트 정규화
  grads /= (K.sqrt(K.mean(K.square(grads))) + 1e-5)

  # 입력 이미지에 대한 손실과 그래디언트를 반환합니다
  iterate = K.function([model.input], [loss, grads])
  
  # 잡음이 섞인 회색 이미지로 시작합니다
  input_img_data = np.random.random((1, size, size, 3)) * 20 + 128.

  # 경사 상승법을 40 단계 실행합니다
  step = 1.
  for i in range(40):
      loss_value, grads_value = iterate([input_img_data])
      input_img_data += grads_value * step
      
  img = input_img_data[0]
  return deprocess_image(img)

In [0]:
### 클래스 활성화의 히트맵 시각화 하기!! -> 클래스 활성화 맵(CAM, Class Activation Map) 시각화
### 1. 이미지의 어느 부분이 컨브넷의 최종 분류 결정에 기여하는지 이해하는 데 유용합니다.
### 2. 분류에 실수가 있는 경우 컨브넷의 결정 과정을 디버깅하는 데 도움이 됩니다.
### 3. 또한 이미지에 특정 물체가 있는 위치를 파악하는 데 사용할 수도 있습니다.

In [0]:
### Grad-CAM: Visual Explanations from Deep Networks via Gradient-based Localization
### 1) 입력 이미지가 주어지면 합성곱 층에 있는 특성 맵의 출력을 추출합니다.
### 2) 그 다음 특성 맵의 모든 채널의 출력에 채널에 대한 클래스의 그래디언트 평균을 곱합니다.
### -> '입력 이미지가 각 채널을 활성화하는 정도’에 대한 공간적인 맵을 '클래스에 대한 각 채널의 중요도’로 가중치를 부여하여 '입력 이미지가 클래스를 활성화하는 정도’에 대한 공간적인 맵을 만드는 것입니다.

In [0]:
# 이전 모든 예제에서는 최상단의 완전 연결 분류기를 제외했지만 여기서는 포함합니다
model = VGG16(weights='imagenet')

이 이미지를 VGG16 모델이 인식할 수 있도록 변환해보죠. 이 모델은 224 × 224 크기의 이미지에서 훈련되었고 keras.applications.vgg16.preprocess_input 함수에 있는 몇 가지 규칙에 따라 전처리 되었습니다. 그러므로 이 이미지를 로드해서 224 × 224 크기로 변경하고 넘파이 float32 텐서로 바꾼 다음 이 전처리 함수를 적용해야 합니다.

In [0]:
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input, decode_predictions
# 224 x 224 크기의 파이썬 이미징 라이브러리(PIL) 객체로 반환된다.
img = image.load_img(img_path, target_size=(224, 224))

# (224, 224, 3) 형태의 NumPy float32 배열
x = image.img_to_array(img)

# 차원을 추가하여 (1, 224, 224, 3) 형태의 배치로 배열을 변환
x = np.expand_dims(x, axis=0)

# 데이터를 전처리합니다(채널별 컬러 정규화를 수행합니다)
x = preprocess_input(x)

preds = model.predict(x)
print('Predicted:', decode_predictions(preds, top=3)[0])
print(np.argmax(preds[0]))

In [0]:
# 예측 벡터의 '고양이' 항목
cat_output = model.output[:, 386]

# VGG16의 마지막 합성곱 층인 block5_conv3 층의 특성 맵
last_conv_layer = model.get_layer('block5_conv3')

# block5_conv3의 특성 맵 출력에 대한 '아프리카 코끼리' 클래스의 그래디언트
grads = K.gradients(cat_output, last_conv_layer.output)[0]

# 특성 맵 채널별 그래디언트 평균 값이 담긴 (512,) 크기의 벡터
pooled_grads = K.mean(grads, axis=(0, 1, 2))

# 샘플 이미지가 주어졌을 때 방금 전 정의한 pooled_grads와 block5_conv3의 특성 맵 출력을 구합니다
iterate = K.function([model.input], [pooled_grads, last_conv_layer.output[0]])

# 두 마리 고양이가 있는 샘플 이미지를 주입하고 두 개의 넘파이 배열을 얻습니다
pooled_grads_value, conv_layer_output_value = iterate([x])

# "고양이" 클래스에 대한 "채널의 중요도"를 특성 맵 배열의 채널에 곱합니다
for i in range(512):
    conv_layer_output_value[:, :, i] *= pooled_grads_value[i]

# 만들어진 특성 맵에서 채널 축을 따라 평균한 값이 클래스 활성화의 히트맵입니다
heatmap = np.mean(conv_layer_output_value, axis=-1)

# 시각화를 위해 히트맵을 0과 1 사이로 정규화하겠습니다. 
heatmap = np.maximum(heatmap, 0)
heatmap /= np.max(heatmap)
plt.matshow(heatmap)
plt.show()



In [0]:
# cv2 모듈을 사용해 원본 이미지를 로드합니다
img = cv2.imread(img_path)

# heatmap을 원본 이미지 크기에 맞게 변경합니다
heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))

# heatmap을 RGB 포맷으로 변환합니다
heatmap = np.uint8(255 * heatmap)

# 히트맵으로 변환합니다
heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)

# 0.4는 히트맵의 강도입니다
superimposed_img = heatmap * 0.4 + img

# 디스크에 이미지를 저장합니다
cv2.imwrite('/content/gdrive/My Drive/Colab Notebooks/datasets/cats_and_dogs_small/cat_cam02.jpg', superimposed_img)